In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [9]:
kospi = fdr.StockListing('KOSPI')
kosdaq = fdr.StockListing('KOSDAQ')
etfs = fdr.StockListing('ETF/KR')

In [7]:
# 날짜는 파이썬 표준라이브러리 datetime 사용
import datetime
# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')

In [8]:
# n개월 전 날짜 계산 함수(개월단위위)
def calculate_start_date(months_ago, end_date):
    start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - relativedelta(months=months_ago)
    return start_date.strftime('%Y-%m-%d')

In [ ]:
kospi.head()

In [ ]:
kospi['Code'].tolist()

In [15]:
kospi.shape

(750, 17)

In [13]:
# 시장과 기간을 입력받아, 모든 종목을 돌면서 수익률을 반환해주는 함수

def return_by_stock(market='KOSPI', month_ago = 1, end_date=today_str):
  
  start_date = calculate_start_date(month_ago, end_date)
  
  results = [] # 수익률 결과를 저장할 리스트
  
  market_list= fdr.StockListing(market)
  
  try:
    
    for stock in market_list['Code'].tolist():
      data = fdr.DataReader(stock, start=start_date, end=end_date)[['Close']]
      
      # 일일 수익률
      data['return'] = data.pct_change()
      
      # NaN 값 처리: 첫 번째 행 삭제
      data = data.dropna()
      
      # 수익률을 퍼센트로 변환
      data['return'] = data['return'] * 100
      
      # 해당 기간 전체 수익률 계산
      total_return = (data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0] * 100

      # 통계 정보 추가
      average_return = data['return'].mean()
      std_dev_return = data['return'].std()
      
      # 결과 저장
      results.append({
          '종목': stock,
          '총 수익률': total_return,
          '평균 수익률': average_return,
          '표준편차': std_dev_return
      })

      # # 결과 출력
      # print(f"종목: {stock}, {month_ago}개월 총 수익률: {total_return:.2f}%, 평균 수익률: {average_return:.2f}%, 표준편차: {std_dev_return:.4f}")
   

    return pd.DataFrame(results)
  except Exception as e:
    print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
    return None


In [ ]:
return_by_stock(3)

In [16]:
# 각 시장의 평균 수익률과 표준편차를 계산하는 함수

def calculate_market_statistics(market='KOSPI', month_ago=1):
    results_df = return_by_stock(market, month_ago)
    if results_df is not None:
      
        # 평균 수익률 및 표준편차 계산
        average_return = results_df['총 수익률'].mean()
        std_dev_return = results_df['총 수익률'].std()
        return average_return, std_dev_return
    return None, None

# 시장별 통계 계산
kospi_avg_return, kospi_std_dev = calculate_market_statistics('KOSPI', 3)
kosdaq_avg_return, kosdaq_std_dev = calculate_market_statistics('KOSDAQ', 3)

# 결과 출력
print("KOSPI 평균 수익률: {:.2f}%, 표준편차: {:.4f}".format(kospi_avg_return, kospi_std_dev))
print("KOSDAQ 평균 수익률: {:.2f}%, 표준편차: {:.4f}".format(kosdaq_avg_return, kosdaq_std_dev))


TypeError: strptime() argument 1 must be str, not int